# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.

## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.

In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [ ]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *
from functions.test_framework.key import generate_bip340_key_pair
from functions.test_framework.address import program_to_witness
from functions.test_framework.script import tagged_hash
from functions.bip0327.reference import individual_pk, key_agg, get_xonly_pk, key_sort, apply_tweak, nonce_gen, nonce_agg, SessionContext, sign, partial_sig_verify, partial_sig_agg, schnorr_verify

import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-09-07T15:35:06.346000Z TestFramework (INFO): PRNG seed is: 6794224631398216675
2025-09-07T15:35:06.347000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_yse43plp


### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
alice_funding_pubkey_b = alice_funding_pubkey.get_bytes()
alice_funding_privkey_b = alice_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey_b)
print(f"Alice funding pubkey: {alice_funding_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_funding_privkey_b.hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
bob_funding_pubkey_b = bob_funding_pubkey.get_bytes()
bob_funding_privkey_b = bob_funding_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey_b)
print(f"Bob funding pubkey: {bob_funding_pubkey_b.hex()}")
print(f"Bob funding privkey: {bob_funding_privkey_b.hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund})
print(f"Transaction creating Alice UTXO: {txid_to_spend}")
rawtx_json = node.getrawtransaction(txid_to_spend, True)
print(json.dumps(rawtx_json, indent=2, default=str))

expected_spk = (b"\x51" + b"\x20" + alice_funding_pubkey_b).hex()
index_to_spend = next(
    v["n"] for v in rawtx_json["vout"]
    if v["scriptPubKey"]["hex"] == expected_spk
)
print(f"Alice transaction to spend: {txid_to_spend}:{index_to_spend}")

result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: 1ec364d5bc1965094cca7b7ab83839e1beb5e959f55b783e0a89358b789199de
Alice funding privkey: 87e862b77d4864ff1ef0abe07f3c78faed2b7fe70cd91ce45fb06427a2d90d67
Alice funding address: bcrt1prmpkf4dur9jsjnx20datswpeuxltt62e74dhs0s23y6ck7y3n80qnmymuk
Bob funding pubkey: 9e930f7edf991b3bc7ce6f4b4db5e97dd7e8ac25340b91f58679f5894d70a61b
Bob funding privkey: a7f0ca98b4b0c3da5a47bfa083f32c8c6e88152914528c92e288347a8987a156
Bob funding address: bcrt1pn6fs7lklnydnh37wda95md0f0ht73tp9xs9eravx086cjnts5cds3nyprx
Transaction creating Alice UTXO: 9e07180d9b0aabdd0f96d1b36c070b6bb5ecb147e13de77fc68c0b08906a5a68
{
  "txid": "9e07180d9b0aabdd0f96d1b36c070b6bb5ecb147e13de77fc68c0b08906a5a68",
  "hash": "61e3b9864b4b06e355711658b270c83569938f91651570df5a52a701a0f197db",
  "version": 2,
  "size": 289,
  "vsize": 208,
  "weight": 829,
  "locktime": 101,
  "vin": [
    {
      "txid": "3516464bda3c7abd1056d5706742df80dcdd2501ac0a3b01659e8faef118da32",
      "vout": 0,
      "scriptSig": {
    

## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [4]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()
alice_change_pubkey_b = alice_change_pubkey.get_bytes()
alice_change_privkey_b = alice_change_privkey.get_bytes()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey_b)
print(f"Alice funding pubkey: {alice_change_pubkey_b.hex()}")
print(f"Alice funding privkey: {alice_change_privkey_b.hex()}")
print(f"Alice funding address: {alice_change_address}")


Alice funding pubkey: 2ec9fe6944d109ef122b99320016cf61a7fc9d9f013b7d814538b25ab59fc040
Alice funding privkey: 390e420afd3197752fe7a9d87501900b9d58b815d50f459f30c31ffde5c7af63
Alice funding address: bcrt1p9mylu62y6yy77y3tnyeqq9k0vxnle8vlqyahmq298ze94dvlcpqqpxwwew


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding musig2 output
privkey_alice_musig2, _ = generate_bip340_key_pair()
pubkey_alice_musig2 = individual_pk(privkey_alice_musig2.get_bytes())
print(f"Alice MuSig2 pubkey: {pubkey_alice_musig2.hex()}")
print(f"Alice MuSig2 privkey: {privkey_alice_musig2.get_bytes().hex()}")

# Create a new Bob key pair for channel funding musig2 output
privkey_bob_musig2, _ = generate_bip340_key_pair()
pubkey_bob_musig2 = individual_pk(privkey_bob_musig2.get_bytes())
print(f"Bob MuSig2 pubkey: {pubkey_bob_musig2.hex()}")
print(f"Bob MuSig2 privkey: {privkey_bob_musig2.get_bytes().hex()}")

tag = "TapTweak"
tweak = sha256(tag.encode('utf-8'))

# Generate a 2-of-2 tweaked aggregate MuSig2 key using the pubkeys form Alice and Bob
sorted_pubkeys = key_sort([pubkey_alice_musig2, pubkey_bob_musig2])
key_agg_ctx = key_agg(sorted_pubkeys)
agg_pubkey = get_xonly_pk(key_agg_ctx)

# Compute the BIP86 tweak
bip86_tweak = tagged_hash("TapTweak", agg_pubkey)
tweaked_ctx = apply_tweak(key_agg_ctx, bip86_tweak, is_xonly=True)

# Final Taproot output key (x-only) to put in the P2TR scriptPubKey
agg_pubkey_tweaked = get_xonly_pk(tweaked_ctx)

print(f"Aggregate musig2 pubkey: {agg_pubkey_tweaked.hex()}")

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked)
print(f"Channel funding musig2 output address: {channel_funding_musig_address}")

Alice MuSig2 pubkey: 021685e2a748d6a780df9c185f0c2a1f2dce53d4f5b635fab3fd657d9c9710b326
Alice MuSig2 privkey: 64ce39eabbc654afe46cd62d685dccd93fe00398b22c9ab762e41e6bb4ac634e
Bob MuSig2 pubkey: 0280cb39014bf8fc48d1217e74a17ad59da616400ae6502ed64f2eafe52ee5a6fe
Bob MuSig2 privkey: dd1c6ff714b609e7ca31086f2c025b365ba47a3e4c0bc7c572779680ad01470c
Aggregate musig2 pubkey: ee2dec0cefced787cf79201da90496510026082f6d92a5632a2654b90064a85f
Channel funding musig2 output address: bcrt1pack7cr80emtc0nmeyqw6jpyk2yqzvzp0dkf22ce2ye2tjqry4p0sr3n4cg


#### The channel funding transaction

##### The unsigned transaction

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
channel_value_sat = int(float("0.01") * 100000000)
channel_value = channel_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
channel_spk = bytes.fromhex("51") + varint_len(agg_pubkey_tweaked) + agg_pubkey_tweaked

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
alice_change_value_sat = int(float(alice_initial_fund) * 100000000) - channel_value_sat - tx_fee_sat
alice_change_value = alice_change_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_change_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey_b) + alice_change_pubkey_b

outputs = (
    channel_value
    + varint_len(channel_spk)
    + channel_spk
    + alice_change_value
    + varint_len(alice_change_spk)
    + alice_change_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001685a6a90080b8cc67fe73de147b1ecb56b0b076cb3d1960fddab0a9b0d18079e0100000000ffffffff0240420f0000000000225120ee2dec0cefced787cf79201da90496510026082f6d92a5632a2654b90064a85f949de605000000002251202ec9fe6944d109ef122b99320016cf61a7fc9d9f013b7d814538b25ab59fc04000000000
{
  "txid": "1172b200525fe9fb0db73926dbb46980a376cb74ac0215c2fb3fab83065a9571",
  "hash": "1172b200525fe9fb0db73926dbb46980a376cb74ac0215c2fb3fab83065a9571",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "9e07180d9b0aabdd0f96d1b36c070b6bb5ecb147e13de77fc68c0b08906a5a68",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.01000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 ee2dec0cefced787cf79201da90496510026082f6d92a5632a2654b90064a85f",
        "desc": "rawtr(ee2dec0cefced787cf79201da90496510026082f6d92a5632a2654b9

##### The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [ ]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index)

input_amount_sat = int(alice_initial_fund * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey_b) + alice_funding_pubkey_b

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
)

sha_sequences = sha256(sequence)

sha_outputs = sha256(outputs)

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode())
sighash = sha256(tag_hash + tag_hash + sig_msg)


##### Signing the sighash

In [8]:
aux_rand = b"\x00"*32
signature = alice_funding_privkey.sign_schnorr(sighash, aux_rand) 

##### The signed transaction

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [9]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

funding_channel_txid = node.sendrawtransaction(signed_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("funding_channel_txid: ", funding_channel_txid)

signed tx:  02000000000101685a6a90080b8cc67fe73de147b1ecb56b0b076cb3d1960fddab0a9b0d18079e0100000000ffffffff0240420f0000000000225120ee2dec0cefced787cf79201da90496510026082f6d92a5632a2654b90064a85f949de605000000002251202ec9fe6944d109ef122b99320016cf61a7fc9d9f013b7d814538b25ab59fc0400140bdd2bf533822fcc6f8cd2bce1c79e486c43db79377ccde967f930abe880b9a7119933361d6aa706966b93eb32a1501709b7e13eaf10c4cb25f113033b1fe0f5e00000000
{
  "txid": "1172b200525fe9fb0db73926dbb46980a376cb74ac0215c2fb3fab83065a9571",
  "hash": "8ae96abf1b67c64ada9464dfb3218619299caccc7026f37968fb0a881efb5b01",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "9e07180d9b0aabdd0f96d1b36c070b6bb5ecb147e13de77fc68c0b08906a5a68",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "bdd2bf533822fcc6f8cd2bce1c79e486c43db79377ccde967f930abe880b9a7119933361d6aa706966b93eb32a1501709b7e13eaf10c4cb25f113033b1f